In [1]:
import scipy.integrate
import numpy as np
import pandas as pd
import math
from scipy.special import jacobi
from scipy.integrate import solve_bvp

In [2]:
n = 6
p = lambda x: (x - 2.0) / (x + 2.0)
q = lambda x: x
r = lambda x: (1.0 - math.sin(x))
f = lambda x: x ** 2

In [3]:

def w(i, x):
    return (1.0 - x ** 2.0) * jacobi(n=i - 1, alpha=1, beta=1)(x)


def d_w(i, x):
    return (-2.0) * i * jacobi(n=i, alpha=0, beta=0)(x)


def dd_w(i, x):
    return -(i + 1.0) * i * jacobi(n=i - 1, alpha=1, beta=1)(x)


def psi_i(i, x):
    return jacobi(n=i - 1, alpha=0, beta=0)(x)


def psi(i):
    return lambda x: jacobi(n=i - 1, alpha=0, beta=0)(x)


# (y, z)
def scalar(y, z):
    result, error = scipy.integrate.quad(lambda x: y(x) * z(x), -1, 1)
    return result


# (Lw_i, psi_j)
def integrate(i, j):
    def und_integral(x):
        return (p(x) * dd_w(i, x) + q(x) * d_w(i, x) + r(x) * w(i, x)) * psi_i(j, x)

    result, error = scipy.integrate.quad(und_integral, -1.0, 1.0)  # integrate(und_integral, -1, 1)
    return result


In [4]:
def moment_method(n):
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            A[i, j] = integrate((j + 1), (i + 1))
    b = np.zeros((n, 1))
    for i in range(n):
        b[i] = scalar(f, psi(i + 1))
    condition_number = np.linalg.norm(A) * np.linalg.norm(np.linalg.inv(A))
    c = np.linalg.solve(A, b)

    def y_n_c(x):
        return sum([float(c[i]) * w(i + 1, x) for i in range(n)])

    return pd.DataFrame(A), pd.DataFrame(b), pd.DataFrame(c), condition_number, y_n_c(-0.5), y_n_c(0), y_n_c(0.5)


In [5]:

def eq(x, y):
    u, v = y
    return np.vstack((v, ((np.sin(x) - 1.0) * u - x * v + x ** 2) * (x + 2.0) / (x - 2.0)))


def bc(ya, yb):
    return np.array([ya[0], yb[0]])


x = np.linspace(-1, 1, 101)
y0 = np.zeros((2, x.size))

res = solve_bvp(eq, bc, x, y0)

In [6]:
t_v = [res.sol(-0.5)[0], res.sol(0)[0], res.sol(0.5)[0]]
columns = ['n', 'mu(A)', 'y_n(-0.5)', 'y_n(0)', 'y_n(0.5)', 'delta(-0.5)', 'delta(0)', 'delta(0.5)']
df_mm = pd.DataFrame(columns=columns)
for n in range(3, 11):
    t = moment_method(n)
    row = {'n': n, 'mu(A)': t[3], 'y_n(-0.5)': t[4], 'y_n(0)': t[5],
           'y_n(0.5)': t[6], 'delta(-0.5)': t[4] - t_v[0],
           'delta(0)': t[5] - t_v[1], 'delta(0.5)': t[6] - t_v[2]}
    df_mm = df_mm.append(row, ignore_index=True)

A, b, c, con, y_1, y_2, y_3 = moment_method(6)

/var/folders/25/hkg7hrfn7130z9vkbc1z1h580000gn/T/ipykernel_2858/2319785543.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mm = df_mm.append(row, ignore_index=True)
/var/folders/25/hkg7hrfn7130z9vkbc1z1h580000gn/T/ipykernel_2858/2319785543.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mm = df_mm.append(row, ignore_index=True)
/var/folders/25/hkg7hrfn7130z9vkbc1z1h580000gn/T/ipykernel_2858/2319785543.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mm = df_mm.append(row, ignore_index=True)
/var/folders/25/hkg7hrfn7130z9vkbc1z1h580000gn/T/ipykernel_2858/2319785543.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  d

In [7]:
A

,0,1,2,3,4,5
0,4.788898,-9.963060,19.450805,-21.099645,33.253984,-31.972727
1,-1.825937,10.400226,-15.105637,26.231643,-26.500706,39.945111
2,-0.461059,-4.131253,16.293716,-20.275313,32.896910,-31.844005
3,0.034185,0.020969,-6.696843,22.207058,-25.422870,39.533442
4,0.018577,-0.127407,0.635971,-9.308806,28.131265,-30.565341
5,-0.008259,0.054332,-0.306383,1.294117,-11.941095,34.063009


In [8]:
b

,0
0,6.666667e-01
1,-7.032712e-18
2,2.666667e-01
3,-2.544265e-17
4,7.865218e-17
5,8.587716e-18


In [9]:
con

62.26829073599271

In [10]:
c

,0
0,0.105456
1,0.047124
2,0.042032
3,0.016705
4,0.007089
5,0.002179


In [11]:
df_mm

,n,mu(A),y_n(-0.5),y_n(0),y_n(0.5),delta(-0.5),delta(0),delta(0.5)
0,3.0,15.707689,0.051149,0.116489,0.184131,-0.0032,0.039033,0.07415
1,4.0,26.854492,0.054563,0.069316,0.125218,0.000214,-0.00814,0.015238
2,5.0,43.293878,0.055412,0.075429,0.108855,0.001063,-0.002027,-0.001125
3,6.0,62.268291,0.053812,0.078362,0.108301,-0.000537,0.000906,-0.001679
4,7.0,85.150096,0.054389,0.077588,0.109557,0.00004,0.000132,-0.000423
5,8.0,110.623404,0.05439,0.077344,0.110009,0.000041,-0.000113,0.000028
6,9.0,140.091636,0.054329,0.077443,0.110032,-0.00002,-0.000013,0.000052
7,10.0,172.153254,0.054351,0.077469,0.109994,0.000002,0.000013,0.000014


Метод коллокации

In [12]:

def omega(x, i):
    return ((1.0 - x ** 2.0) ** 2.0) * jacobi(n=i - 3, alpha=2, beta=2)(x)


def omega_i_x(i, x):
    if i == 1:
        return  1.0 * x ** 2  - 1.0
    elif i == 2:
        return 1.0 * x ** 3 - 1.0 * x
    else:
        return omega(x, i)


def d_omega(x, i):
    return (-2.0) * (i - 2.0) * (1.0 - x ** 2.0) * jacobi(n=i - 2, alpha=1, beta=1)(x)


def d_omega_i_x(i, x):
    if i == 1:
        return 2.0 * x
    elif i == 2:
        return 3.0 * x ** 2 - 1.0
    else:
        return d_omega(x, i)


def dd_omega(x, i):
    return (4.0) * (i - 2.0) * (i - 1.0) * jacobi(n=i - 1, alpha=0, beta=0)(x)


def dd_omega_i_x(i, x):
    if i == 1:
        return 2.0
    elif i == 2:
        return 6.0 * x
    else:
        return dd_omega(x, i)

In [13]:
def collocation_method(n):
    cheb_roots = [math.cos((2.0 * k - 1.0) / (2.0 * n) * math.pi) for k in range(1, n + 1)]
    A_c = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            A_c[i, j] = p(cheb_roots[i]) * dd_omega_i_x((j + 1), (cheb_roots[i])) + q(cheb_roots[i]) * d_omega_i_x(
                (j + 1), (cheb_roots[i])) + r(cheb_roots[i]) * omega_i_x((j + 1), (cheb_roots[i]))
    b_c = np.zeros((n, 1))
    for i in range(n):
        b_c[i] = f(cheb_roots[i])
    c_c = np.linalg.solve(A_c, b_c)
    condition_number = np.linalg.norm(A_c) * np.linalg.norm(np.linalg.inv(A_c))

    def y_n_c(x):
        return sum([float(c_c[i]) * omega_i_x(i + 1, x) for i in range(n)])

    return pd.DataFrame(A_c), pd.DataFrame(b_c), pd.DataFrame(c_c), condition_number, y_n_c(-0.5), y_n_c(0), y_n_c(0.5)

In [14]:
df_cl = pd.DataFrame(columns=columns)
for n in range(3, 11):
    t = collocation_method(n)
    row = {'n': n, 'mu(A)': t[3], 'y_n(-0.5)': t[4], 'y_n(0)': t[5],
           'y_n(0.5)': t[6], 'delta(-0.5)': t[4] - t_v[0],
           'delta(0)': t[5] - t_v[1], 'delta(0.5)': t[6] - t_v[2]}
    df_cl = df_cl.append(row, ignore_index=True)


A, b, c, con, y_1, y_2, y_3 = collocation_method(6)

/var/folders/25/hkg7hrfn7130z9vkbc1z1h580000gn/T/ipykernel_2858/2252061508.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cl = df_cl.append(row, ignore_index=True)
/var/folders/25/hkg7hrfn7130z9vkbc1z1h580000gn/T/ipykernel_2858/2252061508.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cl = df_cl.append(row, ignore_index=True)
/var/folders/25/hkg7hrfn7130z9vkbc1z1h580000gn/T/ipykernel_2858/2252061508.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cl = df_cl.append(row, ignore_index=True)
/var/folders/25/hkg7hrfn7130z9vkbc1z1h580000gn/T/ipykernel_2858/2252061508.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  d

In [15]:
A

,0,1,2,3,4,5
0,1.156837,-0.294371,-2.758152,-7.438083,-12.778316,-17.265440
1,-0.130366,-1.796571,-1.867594,0.621070,8.782026,15.833286
2,-2.101917,-1.583527,2.861427,7.365118,-4.701754,-21.993588
3,-3.632415,2.524675,4.989652,-12.068690,-8.562844,36.888798
4,-4.012491,9.113077,-4.775263,-8.167257,41.110831,-64.978918
5,-3.992453,15.002971,-20.881786,56.043706,-95.543227,127.481874


In [16]:
b

,0
0,0.933013
1,0.500000
2,0.066987
3,0.066987
4,0.500000
5,0.933013


In [17]:
c

,0
0,-0.268442
1,-0.175170
2,-0.204621
3,-0.048192
4,-0.015297
5,-0.004269


In [18]:
con

97.61246412562758

In [19]:
df_cl

,n,mu(A),y_n(-0.5),y_n(0),y_n(0.5),delta(-0.5),delta(0),delta(0.5)
0,3.0,17.349462,0.043976,0.165249,0.29685,-0.010373,0.087792,0.186869
1,4.0,31.876329,0.046045,0.048811,0.121306,-0.008304,-0.028645,0.011326
2,5.0,58.673991,0.056427,0.072268,0.101131,0.002078,-0.005188,-0.008849
3,6.0,97.612464,0.053251,0.079118,0.106307,-0.001098,0.001661,-0.003674
4,7.0,147.828221,0.054373,0.07762,0.109512,0.000024,0.000164,-0.000469
5,8.0,209.117439,0.054433,0.077235,0.110131,0.000084,-0.000221,0.000151
6,9.0,281.694334,0.054314,0.077443,0.110067,-0.000035,-0.000014,0.000086
7,10.0,365.593192,0.054351,0.077479,0.109992,0.000002,0.000022,0.000011
